In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
%matplotlib inline


## Predicting the Genre of Books from Summaries

We'll use a set of book summaries from the [CMU Book Summaries Corpus](http://www.cs.cmu.edu/~dbamman/booksummaries.html) in this experiment.  This contains a large number of summaries (16,559) and includes meta-data about the genre of the books taken from Freebase.  Each book can have more than one genre and there are 227 genres listed in total.  To simplify the problem of genre prediction we will select a small number of target genres that occur frequently in the collection and select the books with these genre labels.  This will give us one genre label per book. 

Your goal in this portfolio is to take this data and build predictive models to classify the books into one of the five target genres.  You will need to extract suitable features from the texts and select suitable models to classify them. You should build and evaluate at least TWO models and compare the prediction results.

You should report on each stage of your experiment as you work with the data.


In [9]:
#Binary Classification
#Multi-class classification
#Multi-label classification

## Data Preparation

The first task is to read the data. It is made available in tab-separated format but has no column headings. We can use `read_csv` to read this but we need to set the separator to `\t` (tab) and supply the column names.  The names come from the [ReadMe](data/booksummaries/README.txt) file.

In [10]:
names = ['wid', 'fid', 'title', 'author', 'date', 'genres', 'summary']

books = pd.read_csv("data/booksummaries/booksummaries.txt", sep="\t", header=None, names=names, keep_default_na=False)
books.head()

,wid,fid,title,author,date,genres,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,,,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


We next filter the data so that only our target genre labels are included and we assign each text to just one of the genre labels.  It's possible that one text could be labelled with two of these labels (eg. Science Fiction and Fantasy) but we will just assign one of those here. 

In [11]:
target_genres = ["Children's literature",
                 'Science Fiction',
                 'Novel',
                 'Fantasy',
                 'Mystery']

# create a Series of empty strings the same length as the list of books
genre = pd.Series(np.repeat("", books.shape[0]))
# look for each target genre and set the corresponding entries in the genre series to the genre label
for g in target_genres:
    genre[books['genres'].str.contains(g)] = g

# add this to the book dataframe and then select only those rows that have a genre label
# drop some useless columns
books['genre'] = genre
genre_books = books[genre!=''].drop(['genres', 'fid', 'wid'], axis=1)

genre_books.shape


(8954, 5)

In [12]:
genre_books.head()

,title,author,date,summary,genre
0,Animal Farm,George Orwell,1945-08-17,"Old Major, the old boar on the Manor Farm, ca...",Children's literature
1,A Clockwork Orange,Anthony Burgess,1962,"Alex, a teenager living in near-future Englan...",Novel
2,The Plague,Albert Camus,1947,The text of The Plague is divided into five p...,Novel
4,A Fire Upon the Deep,Vernor Vinge,,The novel posits that space around the Milky ...,Fantasy
6,A Wizard of Earthsea,Ursula K. Le Guin,1968,"Ged is a young boy on Gont, one of the larger...",Fantasy


In [13]:
# check how many books we have in each genre category
genre_books.groupby('genre').count()


,title,author,date,summary
genre,,,,
Children's literature,1092,1092,1092,1092
Fantasy,2311,2311,2311,2311
Mystery,1396,1396,1396,1396
Novel,2258,2258,2258,2258
Science Fiction,1897,1897,1897,1897


In [39]:
## Label Encoding

In [41]:
from sklearn.preprocessing import LabelEncoder

In [43]:
le = LabelEncoder()
le.fit(list(genre_books['genre'].values))

LabelEncoder()

In [44]:
# create a new column 'genre_no' to store encoded labels
genre_books['genre_no'] = le.transform(list(genre_books['genre']))

In [46]:
# lets see what dataframe looks like
genre_books.head(20)

,title,author,date,summary,genre,genre_no
0,Animal Farm,George Orwell,1945-08-17,"Old Major, the old boar on the Manor Farm, ca...",Children's literature,0
1,A Clockwork Orange,Anthony Burgess,1962,"Alex, a teenager living in near-future Englan...",Novel,3
2,The Plague,Albert Camus,1947,The text of The Plague is divided into five p...,Novel,3
4,A Fire Upon the Deep,Vernor Vinge,,The novel posits that space around the Milky ...,Fantasy,1
6,A Wizard of Earthsea,Ursula K. Le Guin,1968,"Ged is a young boy on Gont, one of the larger...",Fantasy,1
8,Blade Runner 3: Replicant Night,K. W. Jeter,1996-10-01,"Living on Mars, Deckard is acting as a consul...",Science Fiction,4
9,Blade Runner 2: The Edge of Human,K. W. Jeter,1995-10-01,Beginning several months after the events in ...,Science Fiction,4
20,Crash,J. G. Ballard,1973,The story is told through the eyes of narrato...,Novel,3
21,Children of Dune,Frank Herbert,1976,Nine years after Emperor Paul Muad'dib walked...,Science Fiction,4
23,Chapterhouse Dune,Frank Herbert,1985-04,The situation is desperate for the Bene Gesse...,Science Fiction,4


In [48]:
target_names = list(le.inverse_transform([0,1,2,3,4]))
target_names

["Children's literature", 'Fantasy', 'Mystery', 'Novel', 'Science Fiction']

In [ ]:
## TFID Vectorizer 

In [81]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [82]:
vectorizer = TfidfVectorizer(max_features=20000, stop_words = stopwords.words('english'), min_df = 5)
X = vectorizer.fit_transform(genre_books.summary).toarray()
print(X.shape)      

NameError: name 'stopwords' is not defined

In [ ]:
## Split Training

In [57]:
from sklearn.model_selection import train_test_split

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, genre_books['genre_no'], test_size = 0.2, random_state = 142)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7163, 20000)
(1791, 20000)
(7163,)
(1791,)


In [60]:
## Logistic Regression 

In [64]:
from sklearn.linear_model import LogisticRegression

In [66]:
lr = LogisticRegression(solver = 'sag')
lr.fit(X_train, y_train)

LogisticRegression(solver='sag')

In [68]:
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

In [70]:
from sklearn.metrics import classification_report

In [71]:
print("Scores on training set")
print(classification_report(y_train, y_train_pred, target_names = target_names))

Scores on training set
                       precision    recall  f1-score   support

Children's literature       0.91      0.76      0.83       855
              Fantasy       0.89      0.90      0.90      1858
              Mystery       0.94      0.84      0.89      1125
                Novel       0.82      0.91      0.86      1828
      Science Fiction       0.86      0.89      0.87      1497

             accuracy                           0.87      7163
            macro avg       0.89      0.86      0.87      7163
         weighted avg       0.88      0.87      0.87      7163



In [73]:
print("Scores on testing set")
print(classification_report(y_test, y_test_pred, target_names = target_names))

Scores on testing set
                       precision    recall  f1-score   support

Children's literature       0.73      0.45      0.56       237
              Fantasy       0.73      0.79      0.76       453
              Mystery       0.77      0.62      0.69       271
                Novel       0.60      0.80      0.68       430
      Science Fiction       0.77      0.70      0.73       400

             accuracy                           0.70      1791
            macro avg       0.72      0.67      0.68      1791
         weighted avg       0.71      0.70      0.70      1791



In [78]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_test_pred))

[[107  48  11  64   7]
 [ 13 356   6  44  34]
 [  7  19 169  64  12]
 [ 13  21  23 343  30]
 [  6  45  10  59 280]]


## Feature Exaction

Now you take over to build a suitable model and present your results.

Firstly, you need to perform feature extraction to produce feature vectors for the predictive models.

In [49]:
vectorizer = TfidfVectorizer(max_features=20000)
X = vectorizer.fit_transform(genre_books.summary)
print(X.shape)

(8954, 20000)


In [50]:
print(vectorizer.get_feature_names())

['00', '000', '00000', '10', '100', '1000', '108', '10th', '11', '12', '120', '13', '13th', '14', '14th', '15', '150', '16', '16th', '17', '17th', '18', '180', '1863', '1865', '1882', '18th', '19', '1903', '1909', '1912', '1913', '1914', '1916', '1917', '1918', '1919', '1920s', '1928', '1929', '1930s', '1932', '1933', '1934', '1936', '1937', '1938', '1939', '1940', '1940s', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1950s', '1952', '1953', '1954', '1955', '1956', '1958', '1959', '1960', '1960s', '1961', '1962', '1963', '1964', '1966', '1967', '1968', '1969', '1970', '1970s', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1980', '1980s', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1990s', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '19th', '1st', '20', '200', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2050', '20th'

In [51]:
vectorizer = TfidfVectorizer(max_features=20000)
Y = vectorizer.fit_transform(genre_books.genre).toarray()
print(Y.shape)

(8954, 7)


In [52]:
print(vectorizer.get_feature_names())

['children', 'fantasy', 'fiction', 'literature', 'mystery', 'novel', 'science']


## Model Training

Then, train two predictive models from the given data set.

## Model Evaluation

Finally, evaluate and compare the learned predictive models.